<!-- Developed by Rick137Codes -->

# Installation Instructions


<strong>1. Download Anaconda<br><br>
2. Install:</strong>
    <ul>
    <li>pip install selenium
    <li>pip install webdriver_manager
    <li>pip install 2captcha-python
    <li>pip install playsound
    <li>pip install PIL
    </ul>
    
<strong>3. For Execution</strong> <br>
<ul><li>Cell 
    <li>Run All
    <li>Scan Messages Barcode with Android Phone when the page is opened in Chrome browser.

## User Inputs

In [ ]:
#Registered Mobile Number
mobile_number='9494829242'


#Filter to be Applied
filters=['Age 18+','Covishield']

#State
state="Karnataka"

#District
districts=['Dharwad']

#Every conversation on the Google Message web is marked by a unique number in the url.
#Enter the unique number of the conversation in which otps are received.
otp_message_url_number='13'  

#2Captcha API Key to solve captchas in booking page
two_captcha_api_key='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

## Status codes:
- 400 - invalid captcha
- 503 - fully booked 
- 1 - Successfully Booked

In [ ]:
from selenium.webdriver import Chrome
from PIL import Image
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import re
from PIL import Image
import shutil
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import timezone
import os
import ssl

from selenium.webdriver.common.action_chains import ActionChains
from IPython.display import clear_output

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import random
from twocaptcha import TwoCaptcha



def open_browser():
    opts = Options()
    opts.headless = False
    
    
    #driver = webdriver.Chrome(desired_capabilities=capa)
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts) 
    driver.maximize_window()
    return driver






def login_and_send_otp(driver1):
    #driver1.get("https://selfregistration.cowin.gov.in/")
    #time.sleep(2)
    driver1.find_element_by_xpath("//input[@formcontrolname='mobile_number']").send_keys(mobile_number)
    time.sleep(1)
    login_ele=driver1.find_element_by_xpath("//ion-button[contains(text(),'Get OTP')]")
    driver1.execute_script("arguments[0].click();",login_ele)
    
    
def zoom_percent(driver,zoom_percent):
    zoom_text=round(float(zoom_percent)/100,2)
    driver.execute_script(f"document.body.style.zoom='{zoom_text}'")
    
def send_confirmation_message(driver_messages,message):
    driver_messages.get("https://messages.google.com/web/conversations/1658")
    time.sleep(5)
    text_ele=driver_messages.find_element_by_xpath("//textarea[@class='input']")
    #text_ele.send_keys(message)
    time.sleep(.5)
    driver_messages.execute_script(f"arguments[0].value='{message}'",text_ele)
    time.sleep(1)
    text_ele.send_keys(Keys.ENTER)
    time.sleep(.5)
    text_ele.send_keys(Keys.ENTER)
    print("Message Sent!")
    
    
def solve_normalcaptcha(image_path):
    print("Solving Normal Captcha...")
    now1=datetime.now()
    api_key = 'c6e28c18ffcf7ded57a3c7edbab1d58b'
    solver = TwoCaptcha(api_key)
    result=""
    try:
        result = solver.normal(image_path)

    except Exception as e:
        print(e)
    now2=datetime.now()
    print("Time Taken for Solving Captcha:",now2-now1)
    print("Result:",result['code'])
    return result['code']

def save_captcha(driver1,image_url):
    print("Saving Captcha..")
    driver1.switch_to_window(driver1.window_handles[0])
    driver1.execute_script(f'''window.open();''')
    time.sleep(.25)
    driver1.switch_to_window(driver1.window_handles[1])
    time.sleep(.25)
    driver1.get(image_url)
    time.sleep(.5)
    driver1.save_screenshot("Captcha.png")
    
    # Opens a image in RGB mode
    im = Image.open(r"Captcha.png")


    left = 0
    top = 0
    right = 300
    bottom = 100

    # Cropped image of above dimension
    # (It will not change orginal image)
    im1 = im.crop((left, top, right, bottom))
    cropped_image_path="CroppedCaptcha.png"
    im1.save("CroppedCaptcha.png")
    #driver1.close()
    driver1.switch_to_window(driver1.window_handles[0])
    return cropped_image_path


def book_appointment(driver1):
    #Code to book appointment
    try:
        close_all_extra_tabs(driver1)
    except Exception as e:
        print("--------------------------------------")
        print("Exception. | Closing all extra tabs.",e)
        print("--------------------------------------")
    try:
        print("________________________________________________________________")
        print("Centre:\n",driver1.find_element_by_xpath("//div[@class='Hospital-name']").text)
        print("________________________________________________________________")
    except:
        pass
    for image_url_attempt in range(5):
        try:
            image_url=driver1.find_element_by_id("captchaImage").get_attribute('src')
            if(len(str(image_url))>5):
                break
            else:
                time.sleep(1)
        except:
            time.sleep(1)
            continue
    time_element=driver1.find_elements_by_xpath("//ion-button[@class='time-slot ng-star-inserted md button button-solid ion-activatable ion-focusable hydrated']")[-2]
    driver1.execute_script("arguments[0].click();",time_element)
    
    try:
        cropped_captcha_path=save_captcha(driver1,image_url)
    except Exception as e:
        print("Exception | Saving Captcha.",e.args)
        pass
    captcha_result=solve_normalcaptcha(cropped_captcha_path)
    
    driver1.find_element_by_xpath("//input[@placeholder='Enter Security Code']").clear()
    driver1.find_element_by_xpath("//input[@placeholder='Enter Security Code']").send_keys(captcha_result)
    
    submit_button=driver1.find_element_by_xpath("//ion-button[@type='submit']")
    driver1.execute_script("arguments[0].click();",submit_button)
    
    for _ in range(10):
        time.sleep(.25)
        error_message=""
        try:
            error_message=driver1.find_element_by_id("toast-container").get_attribute("textContent")
            if(len(str(error_message))>10):
                break
        except:
            continue

    status_code=0
    print("Error Message:",error_message)
    if(len(str(error_message))<10):
        try:
            error_message=driver1.find_element_by_xpath("//p[@class='ng-star-inserted']").text
        except:
            pass
    if("completely booked" in str(error_message).lower()):
        status_code=503
    elif(("valid security" in str(error_message).lower()) or ("transaction didn't go through" in str(error_message).lower())):
        status_code=400
        
    if("success" in str(driver1.current_url).lower()):
        print("Appointment Successfully Booked!")
        status_code=1
    
    print("Status Code:",status_code)
    return status_code
    
    
def close_all_extra_tabs(driver1):
    #driver1.switch_to.window(driver1.window_handles[0])
    for window_handle in driver1.window_handles[1:]:
        print(window_handle,"Closing.")
        driver1.switch_to.window(window_handle)
        driver1.close()
    driver1.switch_to.window(driver1.window_handles[0])
        
def appointment_booking_success_message(driver1):
    message_body=r"Covid-19 Vaccine Appointment Details:\r\n"
    try:
        centre=soup.find("div",{"class":"valueClass address"}).text.strip()
        message_body+=r"Centre: "+centre+r"\r\n"
    except:
        pass

    try:
        for detail in soup.find_all("ion-col",{"class":"col-padding appointment-details col-space-mobile md hydrated"}):
            message_body+=str(detail.text).strip()+r"\r\n"
    except:
        pass

    try:
        for detail in soup.find_all("div",{"class":"divTableCell"}):
            message_body+=str(detail.text).strip()+" "
    except:
        pass
    print(message_body)
    return message_body

def error_catching_and_sleep(driver1):
    try:
        if("Something went wrong" in driver1.find_element_by_id("toast-container").text):
            print(driver1.find_element_by_id("toast-container").text)
            print("Halting bot for .5 minute. Too many requests.")
            time.sleep(random.uniform(25,35))
    except Exception as e:
        print("Bot detection evasion measure Exception.",e)
        pass

## Open Browser

In [ ]:
try:
    driver_messages.quit()
except:
    pass
driver_messages=open_browser()

In [ ]:
try:
    driver1.quit()
except:
    pass
driver1=open_browser()

## Opening Messages on Web

In [ ]:
driver_messages.get("https://messages.google.com/web/")
time.sleep(10)  # Time to scan barcode
#Cowin messages
driver_messages.get('https://messages.google.com/web/conversations/'+otp_message_url_number)

## Login

In [ ]:
def delete_cache(driver):
    driver.execute_script("window.open('');")
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(.5)
    driver.get('chrome://settings/clearBrowserData') # for old chromedriver versions use cleardriverData
    time.sleep(.5)
    actions = ActionChains(driver) 
    actions.send_keys(Keys.TAB * 7+ Keys.ENTER) # send right combination
    actions.perform()
    time.sleep(3) # wait some time to finish
    driver.close() # close this tab
    driver.switch_to.window(driver.window_handles[0]) # switch back
    print("Cache Cleared")

In [ ]:
def check_is_logged_in():
    try:
        driver1.get("https://selfregistration.cowin.gov.in/dashboard")
    except:
        pass
    time.sleep(1)
    is_proper_dashboard = "dashboard" in str(driver1.current_url).lower()
    if is_proper_dashboard:        
        try:
            driver1.find_element_by_xpath("//span[contains(text(),'XXX-XXX-9242')]")
            is_proper_dashboard = True
        except:
            is_proper_dashboard = False
    return is_proper_dashboard
    

In [ ]:
def login_verify_otp(driver1,driver_messages,otp_message_url_number):
#     if not login_again:
#         logged_in = check_is_logged_in()
#     if not logged_in or login_again:
#         delete_cache(driver1)
#         try:
#             driver1.get("https://selfregistration.cowin.gov.in")
#         except:
#             pass
    try:
        driver1.get("https://selfregistration.cowin.gov.in/dashboard")
    except:
        pass
    time.sleep(1)
    if("dashboard" in str(driver1.current_url).lower()):
        for attempt in range(7):
            try:
                ActionChains(driver1).send_keys(Keys.PAGE_DOWN).perform()
                time.sleep(.5)
                schedule_ele=driver1.find_elements_by_xpath("//span[@class='m-lablename'][contains(text(),'chedule')]")[0]
                driver1.execute_script("arguments[0].click();",schedule_ele)
                break
            except Exception as e:
                pass
            time.sleep(1)
        return 1
    message_web_url='https://messages.google.com/web/conversations/'
    driver_messages.refresh();
    time.sleep(3)
    login_and_send_otp(driver1)
    for attempt in range(3):
        try:
            if("dashboard" in str(driver1.current_url).lower()):
                for attempt in range(7):
                    try:
                        driver1.find_elements_by_xpath("//span[contains(text(),'chedule')]")[0].click()
                        break
                    except Exception as e:
                        pass
                    time.sleep(1)
    
                break
                
            if(otp_message_url_number not in str(driver_messages.current_url)):
                print("Redirecting to Cowin OTP message section.")
                driver_messages.get(message_web_url+otp_message_url_number)
                time.sleep(2)
            if attempt == 1:
                time.sleep(5)
            soup=BeautifulSoup(driver_messages.page_source,'html5lib')
            last_message_text=soup.find_all("div",{"class":"text-msg-content"})[-1].text
            OTP=re.findall(".*?access CoWIN is ([0-9]{4,8}).*?",last_message_text)[0]
            driver1.find_element_by_xpath("//input[@placeholder='Enter OTP']").clear()
            time.sleep(.5)
            driver1.find_element_by_xpath("//input[@placeholder='Enter OTP']").send_keys(OTP)
            time.sleep(1)
            ActionChains(driver1).send_keys(Keys.ESCAPE).perform()
            ActionChains(driver1).send_keys(Keys.PAGE_DOWN).perform()
            verify_ele=driver1.find_element_by_xpath("//ion-button[contains(text(),'Verify')]")
            driver1.execute_script("arguments[0].click();",verify_ele)
            #driver1.find_element_by_xpath("//div[@class='covid-button-desktop ion-text-center']").click()
            print("Login Initiated.")
            print(f"Waiting for OTP... | {driver1.current_url}")
            print("Last OTP:",OTP)
            time.sleep((attempt*3)+7)
        except Exception as e:
            print("Exception | OTP BYPASS SECTION |",e)

            time.sleep(2)
            continue
    return 1

## Schedule + Inputting Filters

In [ ]:
def schedule_plus_filters(driver1,state,district,filters):
    driver1.get("https://selfregistration.cowin.gov.in/appointment")
    time.sleep(1)
    #driver1.find_element_by_xpath("//input[@appinputchar='pincode']").send_keys(pincode)
    #driver1.find_element_by_xpath("//ion-button[@class='pin-search-btn md button button-solid ion-activatable ion-focusable hydrated']").click()

    #Search by district
    district_search=driver1.find_element_by_xpath("//button[@data-checked='Search By District']")
    time.sleep(.5)
    driver1.execute_script("arguments[0].click();",district_search)
    driver1.find_element_by_xpath("//mat-select[@formcontrolname='state_id']").send_keys(state)
    time.sleep(1)
    driver1.find_element_by_xpath("//mat-select[@formcontrolname='district_id']").send_keys(district)
    time.sleep(.5)
    search_ele=driver1.find_element_by_xpath("//ion-button[@class='pin-search-btn district-search md button button-solid ion-activatable ion-focusable hydrated']")
    driver1.execute_script("arguments[0].click();",search_ele)
    #time.sleep(1)
    
    #Filters
    print("Filters: ",end="")
    for cowin_filter in filters:
        filter_ele=driver1.find_element_by_xpath(f"//label[contains(text(),'{cowin_filter}')]")
        driver1.execute_script("arguments[0].click();",filter_ele)
        print(cowin_filter,end=" | ")
        time.sleep(.25)
    print("")
    #driver1.find_element_by_xpath("//label[contains(text(),'Covaxin')]").click()
    

In [ ]:
def click_open_slot(driver1):
    zoom_percent(driver1,80)
    appointment_found=0
    ActionChains(driver1).send_keys(Keys.PAGE_DOWN).perform()
    time.sleep(.5)
    for vaccine_div in driver1.find_elements_by_xpath("//div[@class='vaccine-box vaccine-box1 vaccine-padding']"):
        #print(vaccine_div.text,"\n")
        if(("Booked" not in vaccine_div.text) and (len(vaccine_div.text.strip())>2)):
            print("Suitable Appointment Found!")
            appointment_found=1
            print(vaccine_div.text,"\n")
            #vaccine_div.click()
            driver1.execute_script("arguments[0].click();",vaccine_div)
            break
    if(appointment_found==0):
        print("No Open Appointments.")
    return appointment_found

## Main

In [ ]:
iteration=0
appointment_found=0
close_all_extra_tabs(driver1)
appointment_booked=0
while(appointment_booked==0):
    now1=datetime.now()
    iteration+=1
    print("#Iteration:",iteration)
    
    for district in districts:
        schedule_success=0
        if(iteration%50==0):
            clear_output()
        
        print(f"{state} | {district}")
        try:
            driver1.find_element_by_xpath("//h2[contains(text(),'Register')]")
            delete_cache(driver1)
            login_verify_otp(driver1,driver_messages,otp_message_url_number)
            time.sleep(2)
        except:
            pass
        try:
            if "dashboard" in str(driver1.current_url).lower():
                driver1.find_element_by_xpath("//span[contains(text(),'XXX-XXX-9242')]")
            schedule_plus_filters(driver1,state,district,filters)
            appointment_found=click_open_slot(driver1)
            if(appointment_found==1):
                break
            time.sleep(random.uniform(3,3.1))
        except Exception as e:
            print("Exception while Scheduling/Clicking Open Slot:",e)
            error_catching_and_sleep(driver1)
            try:
                login_verify_otp(driver1,driver_messages,otp_message_url_number)
            except Exception as e:
                print("Exception Logging in.|",e)
            time.sleep(5)
    now2=datetime.now()
    print("Time taken for iteration:",now2-now1)
    
    if(appointment_found==1):
        for booking_attempt in range(5):
            print("Booking Attempt:",booking_attempt+1)
            try:
                playsound("Booking.mp3")
                status_code=book_appointment(driver1)
            except:
                status_code=400
                
            if(status_code==1):
                appointment_booked=1
                break
            elif(status_code==400):
                continue
            elif(status_code==503):
                print("Fully booked, Resuming Search!")
                break
            time.sleep(.5)
    print("_______________________________________________________________________")

In [ ]:
try:
    os.remove("Alert.mp3")
except:
    pass
from gtts import gTTS
gTTS("Appointment Booked!").save("Alert.mp3")
from playsound import playsound
playsound("Alert.mp3")